## Tidy Data and OPS Calculation

* The goal of this exercise is to find the top 5 seasons according to OPS of the following players 
    * Andrew McCutchen
    * Elvis Andrus
    * Ian Kinsler
    * Jose Altuve
    * Kyle Seager
    * Robinson Cano

In [1]:
import pandas as pd
import numpy as np

In [2]:
hit_piv = pd.read_csv('hitters_hw_prep.csv')

In [3]:
hit_piv.head(1)

,nameFull,H_2012,H_2013,H_2014,H_2015,2B_2012,2B_2013,2B_2014,2B_2015,3B_2012,...,HBP_2014,HBP_2015,SH_2012,SH_2013,SH_2014,SH_2015,SF_2012,SF_2013,SF_2014,SF_2015
0,Andrew McCutchen,194.0,185.0,172.0,165.0,29.0,38.0,38.0,36.0,6.0,...,10.0,12.0,0.0,0.0,0.0,0.0,5.0,4.0,6.0,9.0


## To begin tidying our data, we can use the melt function

* Remember the rules of Tidy Data ([Link](http://vita.had.co.nz/papers/tidy-data.html))
    * Each variable forms a column
    * Each observation forms a row
    * Each type of observational unit forms a table

In [4]:
melt_df = pd.melt(hit_piv, id_vars='nameFull')

In [5]:
melt_df.head()

,nameFull,variable,value
0,Andrew McCutchen,H_2012,194.0
1,Elvis Andrus,H_2012,NaN
2,Elvus Andris,H_2012,180.0
3,Ian Kinsler,H_2012,168.0
4,Jose Altuve,H_2012,167.0


In [6]:
melt_df['nameFull'].value_counts()

Elvus Andris        36
Ian Kinsler         36
Jose Altuve         36
Kyle Seager         36
Andrew McCutchen    36
Elvis Andrus        36
Robinson Cano       36
Name: nameFull, dtype: int64

## To start we can see that Elvis Andrus' name is misspelled in one of the rows

In [7]:
melt_df.loc[melt_df['nameFull']=='Elvus Andris', 'nameFull'] = 'Elvis Andrus'

## To complete tidying our dataframe, we need to split the "variable" column into two columns
* Our dataset still violates the tidy data principles as not every variable forms a column

In [8]:
melt_df['column'] = melt_df['variable'].apply(lambda x: x.split('_')[0])
melt_df['yearID'] = melt_df['variable'].apply(lambda x: x.split('_')[1])

In [9]:
melt_df.head()

,nameFull,variable,value,column,yearID
0,Andrew McCutchen,H_2012,194.0,H,2012
1,Elvis Andrus,H_2012,NaN,H,2012
2,Elvis Andrus,H_2012,180.0,H,2012
3,Ian Kinsler,H_2012,168.0,H,2012
4,Jose Altuve,H_2012,167.0,H,2012


## We can then pivot our data using pandas pivot table to be able to perform operations on our variables

In our case, each player/season is an observation, and our variables are the offensive variables needed to calcuate OPS

* Each variable is a column
* Each observation is a row

In [10]:
pivot_hw_df = pd.pivot_table(melt_df, index=['nameFull','yearID'], columns='column', values='value').reset_index()

In [11]:
pivot_hw_df.head()

column,nameFull,yearID,2B,3B,AB,BB,H,HBP,HR,SF,SH
0,Andrew McCutchen,2012,29.0,6.0,593.0,70.0,194.0,5.0,31.0,5.0,0.0
1,Andrew McCutchen,2013,38.0,5.0,583.0,78.0,185.0,9.0,21.0,4.0,0.0
2,Andrew McCutchen,2014,38.0,6.0,548.0,84.0,172.0,10.0,25.0,6.0,0.0
3,Andrew McCutchen,2015,36.0,3.0,566.0,98.0,165.0,12.0,23.0,9.0,0.0
4,Elvis Andrus,2012,31.0,9.0,629.0,57.0,180.0,5.0,3.0,3.0,17.0


## Before we perform any math, we need to validate that we don't have any issues with our data

* Using the describe we can get a quick description of the distribution of our data

In [12]:
pivot_hw_df.describe().round(3)

column,2B,3B,AB,BB,H,HBP,HR,SF,SH
count,23.000,24.000,24.000,24.000,24.000,24.000,24.000,24.000,24.000
mean,35.304,3.250,554.792,54.375,176.333,6.042,28.958,5.167,3.000
std,6.581,2.289,260.684,16.998,17.613,2.678,64.689,1.834,4.845
min,17.000,0.000,-660.000,29.000,151.000,2.000,2.000,2.000,0.000
25%,31.500,1.750,588.250,43.000,164.500,4.750,7.000,4.000,0.000
50%,35.000,3.000,610.000,52.000,174.500,5.500,18.000,5.000,1.000
75%,39.000,4.250,624.500,62.000,187.250,8.000,23.500,6.000,3.250
max,48.000,9.000,684.000,98.000,225.000,12.000,330.000,9.000,17.000


### Some standout issues

* ABs have a mininum value of -660. As these are all counting stats, there shouldn't be negative values.

* HRs have a maximum value of 330. The record for homeruns in one year is 73, so this is clearly incorrect.

In [13]:
pivot_hw_df[pivot_hw_df['AB']==-660]

column,nameFull,yearID,2B,3B,AB,BB,H,HBP,HR,SF,SH
14,Jose Altuve,2014,47.0,3.0,-660.0,36.0,225.0,5.0,7.0,5.0,1.0


In [14]:
pivot_hw_df.query("HR==330")

column,nameFull,yearID,2B,3B,AB,BB,H,HBP,HR,SF,SH
20,Robinson Cano,2012,48.0,1.0,627.0,61.0,196.0,7.0,330.0,2.0,0.0


## We can overwrite these erroneous values using .loc 
https://pandas.pydata.org/pandas-docs/stable/cookbook.html

* Altuve had 660 ABs in 2014
    * http://m.mlb.com/player/514888/jose-altuve

* Cano hit 33 HRs in 2012
    * http://m.mlb.com/player/429664/robinson-cano

In [15]:
pivot_hw_df.loc[(pivot_hw_df['nameFull']=='Jose Altuve')&
           (pivot_hw_df['yearID']=='2014'), 'AB'] = 660.0

pivot_hw_df.loc[(pivot_hw_df['nameFull']=='Robinson Cano')&
           (pivot_hw_df['yearID']=='2012'), 'HR'] = 33

## We can use the isnull method to determine if we have any null values

In [16]:
pivot_hw_df.isnull().sum()

column
nameFull    0
yearID      0
2B          1
3B          0
AB          0
BB          0
H           0
HBP         0
HR          0
SF          0
SH          0
dtype: int64

In [17]:
pivot_hw_df[pivot_hw_df['2B'].isnull()]

column,nameFull,yearID,2B,3B,AB,BB,H,HBP,HR,SF,SH
19,Kyle Seager,2015,NaN,0.0,623.0,54.0,166.0,5.0,26.0,4.0,0.0


## Kyle Seager had 37 2Bs in 2015
* Source: http://m.mlb.com/player/572122/kyle-seager

In [18]:
pivot_hw_df.loc[(pivot_hw_df['nameFull']=='Kyle Seager')&
           (pivot_hw_df['yearID']=='2015'), '2B'] = 37

## Create Plate Appearances Field

$PA = AB + BB + HBP + SF$

* Lambda function allows us to perform row-wise operations

In [19]:
pivot_hw_df['PA'] = pivot_hw_df.apply(lambda row: row['AB']+row['BB']+row['HBP']+row['SF'], axis=1)

## Create OBP 

$OBP = (H + BB + HBP) / PA$

* We can also perform basic row-wise math using the pandas Series values sliced by column names

In [20]:
pivot_hw_df['OBP'] = (pivot_hw_df['H']+pivot_hw_df['BB']+pivot_hw_df['HBP']) / pivot_hw_df['PA'] 

## Create SLG

$SLG = ((1B) + (2 * 2B) + (3 * 3B) + (4 * HR)) / AB$

We'll first need to create a singles field

In [21]:
pivot_hw_df['1B'] = pivot_hw_df['H'] - (pivot_hw_df['2B'] + pivot_hw_df['3B'] + pivot_hw_df['HR'])

### We can define a function that takes each row to clean up our code

In [22]:
def calc_slg(row):
    db = 2*row['2B']
    tb = 3*row['3B']
    hr = 4*row['HR']
    slg = (row['1B'] + db + tb + hr) / row['AB']
    return slg

In [23]:
pivot_hw_df['SLG'] = pivot_hw_df.apply(lambda row: calc_slg(row), axis=1)

## Calc On-base plus slugging
* $OPS = OBP + SLG$

In [24]:
pivot_hw_df['OPS'] = pivot_hw_df['OBP'] + pivot_hw_df['SLG']

## To get the top 5 seasons by OPS of these players from 2012-2015, we can sort the values in descending order

In [25]:
(pivot_hw_df
 .sort_values('OPS', ascending=False)
 .head()
 .round(3)
 .reset_index(drop=True)
[['nameFull','yearID','OBP','SLG','OPS']])

column,nameFull,yearID,OBP,SLG,OPS
0,Andrew McCutchen,2012,0.400,0.553,0.953
1,Andrew McCutchen,2014,0.410,0.542,0.952
2,Robinson Cano,2012,0.379,0.550,0.929
3,Andrew McCutchen,2013,0.404,0.508,0.911
4,Robinson Cano,2013,0.383,0.516,0.899
